In [1]:
!pip install kaggle

In [ ]:
#downloading the dataset
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

In [ ]:
#Extract the data from zip file
from zipfile import ZipFile
with ZipFile(r"C:\Users\DELL\orders.csv.zip", 'r') as zip_orders:
    zip_orders.extractall(r"C:\Users\DELL\Documents") 

In [ ]:
#read the data from the dataset
import pandas as pd
df = pd.read_csv(r"C:\Users\DELL\Documents\orders.csv")
df

In [ ]:
#data cleaning
#drop duplicates
df.drop_duplicates(inplace=True)
df

In [ ]:
# null check
df1=df.isnull().sum()
df1

In [ ]:
#replace the null value in the ship mode column
if "Ship Mode" in df.columns:
    df["Ship Mode"].fillna(df["Ship Mode"].mode()[0], inplace=True)
df

In [ ]:
#replace the columns name in lowercase and remove the space
df.columns = df.columns.str.lower().str.replace(" ", "_", regex=True)
df

In [ ]:
#revome the space in the data of each row
df=df.apply(lambda x:x.str.strip() if x.dtype=="object" else x)
df

In [ ]:
#creating three new columns and filling the column with respective data
df["discount_price"]=df["list_price"]*(df["discount_percent"]/100)
df["sale_price"]=df["list_price"] - df["discount_price"]
df["profit"]=df["sale_price"] - df["cost_price"]
df

In [ ]:
# Connecting jupter with mysql and creating a database
import pymysql
myconnection = pymysql.connect(host='127.0.0.1',user='root',passwd='Suveeswar@06')
myconnection.cursor().execute("create database retailanalysis") 

In [ ]:
#creating the columns of the table
dtypes_mapping={
    'int64':'INTEGER',
    'object':'TEXT',
    'float64':'FLOAT',
    'datetime64[ns]':'DATETIME'
}
columns=','.join(f"{col} {dtypes_mapping[str(dtypes)]}" for col, dtypes in zip(df.columns,df.dtypes))
query= f"CREATE TABLE RETAILANALYSIS.ORDERS ({columns})"
myconnection.cursor().execute(query)

In [ ]:
#filling the table with their respective values in the dataset
for i in range(0,len(df)):
    myconnection.cursor().execute(f"insert into retailorders.orders values {tuple(df.iloc[i])}")
    myconnection.commit()

In [ ]:
#spliting the table into two halves
#first 12 columns as first table
df_table1=df.iloc[:,0:12]
df_table1

In [ ]:
#splitting last 7 columns with the primary key of the table 1 as the second table
df_table2=df.iloc[:,[0]+ list(range(12,df.shape[1]))]
df_table2

In [ ]:
#creating the columns of the table 1
dtypes_mapping={
    'int64':'INTEGER',
    'object':'TEXT',
    'float64':'FLOAT',
    'datetime64[ns]':'DATETIME'
}
column_definitions = [
    f"{col} {dtypes_mapping[str(dtype)]}" for col, dtype in zip(df_table1.columns, df_table1.dtypes)
]

# Define Primary Key
primary_key = "order_id"

# Construct the SQL query correctly
query = f"""
CREATE TABLE RETAILANALYSIS.RODETAILS (
    {', '.join(column_definitions)},
    PRIMARY KEY ({primary_key})
);
"""
print(query)

# Execute the SQL Query
cursor = myconnection.cursor()
cursor.execute(query)
myconnection.commit()

In [ ]:
#importing the values of the table1 from the dataframe
for i in range(0,len(df_table1)):
    myconnection.cursor().execute(f"insert into retailanalysis.rodetails values {tuple(df_table1.iloc[i])}")
    myconnection.commit()

In [ ]:
### creating the columns of the table 2
dtypes_mapping={
    'int64':'INTEGER',
    'object':'TEXT',
    'float64':'FLOAT',
    'datetime64[ns]':'DATETIME'
}
column_definitions = [
    f"{col} {dtypes_mapping[str(dtype)]}" for col, dtype in zip(df_table2.columns, df_table2.dtypes)
]

foreign_key = "order_id"  # Set the Foreign Key column
references_table = "RODETAILS"

# Construct the SQL query correctly
query = f"""
CREATE TABLE RETAILANALYSIS.ROPRICE (
    {', '.join(column_definitions)},
    FOREIGN KEY ({foreign_key}) REFERENCES {references_table}({foreign_key})
);
"""
print(query)

# Execute the SQL Query
myconnection.cursor().execute(query)
myconnection.commit()

In [ ]:
#importing the values of the table 2 from the dataframe
for i in range(0,len(df_table2)):
    myconnection.cursor().execute(f"insert into retailanalysis.roprice values {tuple(df_table2.iloc[i])}")
    myconnection.commit()

In [ ]:
# Create a single cursor object
cursor = myconnection.cursor()

# Define the SQL query for Find top 10 highest revenue generating products
query1 = """
SELECT sub_category, round(SUM(sale_price * quantity)) AS total_revenue
FROM retailorders.orders
GROUP BY sub_category
ORDER BY total_revenue DESC
LIMIT 10;
"""

# Execute the query
cursor.execute(query1)

# Fetch results
top_products = cursor.fetchall()# Use the same cursor for fetching

# Print the results
for row in top_products:
    print(row)

In [ ]:
#Define the SQL query for Find the top 5 cities with the highest profit margins
query2="""
SELECT city, round(sum(profit*quantity) / SUM(sale_price * quantity) * 100,2) as PROFIT_MARGIN
FROM retailorders.orders 
GROUP BY city
ORDER BY  PROFIT_MARGIN DESC 
LIMIT 5;
"""
cursor.execute(query2)

# Fetch results
profit_cities = cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in profit_cities :
    print(row)

In [ ]:
#Define the SQL query for Calculate the total discount given for each category
query3="""
SELECT category,sub_category, round(sum(discount_price*quantity),2) as tot_discount 
FROM retailorders.orders 
GROUP BY category,sub_category
ORDER BY tot_discount DESC;
"""
cursor.execute(query3)

# Fetch results
total_discount = cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in total_discount :
    print(row)

In [ ]:
#Define the SQL query for Find the average sale price per product category
query4="""
SELECT sub_category, round(avg(sale_price),2) as avg_sale_price 
FROM retailorders.orders 
GROUP BY sub_category
ORDER BY avg_sale_price DESC;
"""
cursor.execute(query4)

# Fetch results
avg_sale_price= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in avg_sale_price :
    print(row)

In [ ]:
#Define the SQL query for Find the region with the highest average sale price
query5="""
SELECT region, round(avg(sale_price),2) as avg_sale_price_per_region
FROM retailorders.orders 
GROUP BY region
ORDER BY avg_sale_price_per_region  DESC
lIMIT 1;
"""
cursor.execute(query5)

# Fetch results
avg_sale_price_per_region= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in avg_sale_price_per_region :
    print(row)


In [ ]:
#Define the SQL query for Find the total profit per category
query6="""
SELECT category, round(sum(profit*quantity),2) as tot_profit_per_category
FROM retailorders.orders 
GROUP BY category
ORDER BY tot_profit_per_category DESC;
"""
cursor.execute(query6)

# Fetch results
tot_profit_per_category= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in tot_profit_per_category :
    print(row)

In [ ]:
#Define the SQL query for Identify the top 3 segments with the highest quantity of orders
query7="""
SELECT segment,cast(sum(quantity) as signed) as tot_quantity
FROM retailorders.orders 
GROUP BY segment
ORDER BY tot_quantity DESC
LIMIT 3;
"""
cursor.execute(query7)

# Fetch results
tot_quantity= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in tot_quantity :
    print(row)

In [ ]:
#Define the SQL query for Determine the average discount percentage given per region
query8="""
SELECT region,CAST(avg(discount_percent) AS FLOAT) as avg_dis_percent
FROM retailorders.orders 
GROUP BY region
ORDER BY avg_dis_percent;
"""
cursor.execute(query8)

# Fetch results
avg_dis_percent= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in avg_dis_percent :
    print(row)

In [ ]:
#Define the SQL query for Find the product category with the highest total profit
query9="""
SELECT category,sub_category,round(sum(profit*quantity),2) as high_profit
FROM retailorders.orders 
GROUP BY category, sub_category
ORDER BY high_profit DESC
LIMIT 1;
"""
cursor.execute(query9)

# Fetch results
high_profit= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in high_profit :
    print(row)


In [ ]:
#Define the SQL query for Calculate the total revenue generated per year
query10="""
SELECT year(order_date) as years,round(sum(sale_price*quantity),2) as rev_per_year
FROM retailorders.orders 
GROUP BY  years 
ORDER BY years DESC;
"""
cursor.execute(query10)

# Fetch results
rev_per_year= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in rev_per_year :
    print(row)

In [ ]:
# Define the SQL query for  Find all order details for the first quater of the year 2022
query11="""
SELECT distinctrow a.order_id, a.order_date,a.ship_mode,a.segment,a.country, a.city,a.state,a.postal_code,a.region,a.sub_category,a.category, round(b.profit*b.quantity) as tot_profit 
FROM retailorders.rodetails a  
JOIN retailorders.roprice b ON a.order_id = b.order_id
WHERE a.order_date BETWEEN '2022-01-01' AND '2022-04-31' ;
"""
cursor.execute(query11)

# Fetch results
join1= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in join1 :
    print(row)

In [ ]:
# Define the SQL query for Find the highest profit product for each category in each state and region.
query12="""
SELECT a.state,a.region,a.category,ROUND(SUM(b.profit * b.quantity),2) as profit_amt 
from retailorders.rodetails a  
LEFT JOIN retailorders.roprice b 
ON a.order_id = b.order_id 
GROUP BY a.state,a.region,a.category 
ORDER BY profit_amt DESC ;
"""
cursor.execute(query12)

# Fetch results
join2= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in join2 :
    print(row)

In [ ]:
# Define the SQL query to Find the profit range (min and max) per month for each year.
query13="""
SELECT year(a.order_date) as year,month(a.order_date) as month,round(sum(b.profit*b.quantity)) as monthly_profit,
ROUND(MIN(b.profit * b.quantity), 2) AS min_profit,
ROUND(MAX(b.profit * b.quantity), 2) AS max_profit
from retailorders.rodetails a  
inner JOIN retailorders.roprice b 
ON a.order_id = b.order_id 
GROUP BY month,year
ORDER BY year asc,month asc;
"""
cursor.execute(query13)

# Fetch results
join3= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in join3:
    print(row)


In [ ]:
# Define the SQL query to find which segment with the highest total quantity of orders
query14 ="""
select a.segment, cast(sum(b.quantity) as float) as tot_quan 
from retailorders.rodetails a  
inner JOIN retailorders.roprice b 
ON a.order_id = b.order_id 
GROUP BY a.segment 
ORDER BY tot_quan desc;
"""
cursor.execute(query14)

# Fetch results
join4= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in join4:
    print(row)

In [ ]:
# Define the SQL query to find the category and sub-category with the highest total quantity and its max discount percentage"
query15="""
SELECT 
    A.CATEGORY, 
    A.SUB_CATEGORY, 
    MAX(B.DISCOUNT_PERCENT) AS MAX_DISCOUNT
FROM retailorders.rodetails A  
JOIN retailorders.roprice B 
ON A.ORDER_ID = B.ORDER_ID 
WHERE (A.CATEGORY, A.SUB_CATEGORY) IN (
    SELECT CATEGORY, SUB_CATEGORY 
    FROM retailorders.rodetails 
    GROUP BY CATEGORY, SUB_CATEGORY  
    ORDER BY SUM(QUANTITY) DESC  
    LIMIT 1 )
GROUP BY A.CATEGORY, A.SUB_CATEGORY;
"""

cursor.execute(query15)

# Fetch results
join5= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in join5:
    print(row)

In [ ]:
# Define the SQL query to Rank product sub-categories based on total revenue
query16="""
WITH ProductRevenue AS (
    SELECT  
        SUB_CATEGORY, 
        SUM(sale_price * quantity) AS total_revenue
    FROM retailorders.ORDERS
    GROUP BY SUB_CATEGORY
)
SELECT 
    SUB_CATEGORY, 
    total_revenue,
    ROW_NUMBER() OVER (ORDER BY total_revenue DESC) AS revenue_rank
FROM ProductRevenue;
"""
cursor.execute(query16)

# Fetch results
join6= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in join6:
    print(row)


In [ ]:
# Define the SQL query to Classify product sub-categories based on total revenue performance
query17="""
WITH ProductRevenue AS (
    SELECT 
        SUB_CATEGORY, 
        SUM(sale_price * quantity) AS total_revenue
    FROM RETAILORDERS.ORDERS
    GROUP BY SUB_CATEGORY
)
SELECT 
    SUB_CATEGORY, 
    total_revenue,
    CASE 
        WHEN total_revenue > 100000 THEN 'High Performer'
        WHEN total_revenue BETWEEN 50000 AND 100000 THEN 'Medium Performer'
        ELSE 'Low Performer'
    END AS performance_category
FROM ProductRevenue;
"""
cursor.execute(query17)

# Fetch results
join7= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in join7:
    print(row)

In [ ]:
# Define the SQL query to Find average discount, total sales, and revenue lost by category and sub-category (for discounts > 3%)
query18="""
SELECT a.CATEGORY,
    a.SUB_CATEGORY, 
    CAST(AVG(b.discount_percent)AS FLOAT) AS avg_discount,
    CAST(ROUND(SUM(b.sale_price * b.quantity),2) AS FLOAT) AS total_sales,
    CAST(ROUND(SUM((b.sale_price * b.quantity) * (b.discount_percent / 100)),2) AS FLOAT) AS revenue_lost_due_to_discount
FROM retailorders.rodetails as a
LEFT JOIN retailorders.roprice as b on a.order_id = b.order_id
WHERE discount_percent > 3
GROUP BY CATEGORY,SUB_CATEGORY 
ORDER BY revenue_lost_due_to_discount DESC;
"""
cursor.execute(query18)

# Fetch results
join8= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in join8:
    print(row)


In [ ]:
# Define the SQL query Find the date(s) with the highest total revenue
query19="""
SELECT 
    a.order_date, 
    round(SUM(b.quantity * b.sale_price),2) AS total_revenue
FROM retailorders.rodetails a
JOIN retailorders.roprice b ON a.order_id = b.order_id  
GROUP BY a.order_date
ORDER BY total_revenue DESC
LIMIT 1;  
"""
cursor.execute(query19)

# Fetch results
join9= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in join9:
    print(row)


In [ ]:
# Define the SQL query Retrieve the top 10 highest spending orders
query20="""
SELECT distinct
    a.order_id,a.order_date,a.ship_mode,a.country,a.city,a.state,a.region,a.category,a.sub_category,
    round(SUM(b.quantity * b.sale_price),2) AS total_spent
FROM retailorders.rodetails as a
JOIN retailorders.roprice as b ON a.order_id = b.order_id 
GROUP BY a.order_id,a.order_date,a.ship_mode,a.country,a.city,a.state,a.region,a.category,a.sub_category
ORDER BY total_spent DESC
LIMIT 10;
"""
cursor.execute(query20)

# Fetch results
join10= cursor.fetchall()  # Use the same cursor for fetching

# Print the results
for row in join10:
    print(row)